In [1]:
import os

In [2]:
%pwd

'd:\\Data Science Projects\\Phishig_Domain\\Phishing-Domain\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Data Science Projects\\Phishig_Domain\\Phishing-Domain'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [8]:
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size
import pandas as pd
import shutil

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
        
            
        # if not os.path.exists(self.config.local_data_file):
        #     # If the file doesn't exist, log an error or handle it
        #     logger.error(f"File not found at {self.config.local_data_file}")
        # else:
        #     # File exists, log and proceed with loading the CSV or other operations
        #     logger.info(f"File already exists at: {self.config.local_data_file}")
            
        #     # Load the CSV as needed
        #     df = pd.read_csv(self.config.source_URL)
        #     logger.info(f"Data loaded successfully from {self.config.local_data_file}")



           # Copy the downloaded file to the new path (unzip path)
        self.save_to_unzip_path()

    def save_to_unzip_path(self):
        """
        Saves the downloaded file to the specified unzip path.
        """
        # Ensure the unzip directory exists
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        
        # Define the destination path
        destination_path = os.path.join(unzip_path, os.path.basename(self.config.local_data_file))
        
        # Copy the file if it doesn't already exist at the destination
        if not os.path.exists(destination_path):
            shutil.copy(self.config.local_data_file, destination_path)
            logger.info(f"File copied to {destination_path}")
        else:
            logger.info(f"File already exists at {destination_path}, skipping copy.")

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.save_to_unzip_path()
except Exception as e:
    raise e

[2024-10-30 23:49:05,152: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-10-30 23:49:05,155: INFO: common: yaml file: params.yaml loaded successfully]
[2024-10-30 23:49:05,155: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-10-30 23:49:05,155: INFO: common: created directory at: artifacts]
[2024-10-30 23:49:05,155: INFO: common: created directory at: artifacts/data_ingestion]
[2024-10-30 23:49:05,928: INFO: 3296042008: artifacts/data_ingestion/data.zip download! with following info: 
Date: Wed, 30 Oct 2024 18:19:07 GMT
Content-Type: text/html; charset=utf-8
Content-Length: 432375
Connection: close
Vary: Accept-Encoding
Vary: Accept-Encoding
Vary: Accept-Encoding
Vary: Accept-Encoding
Cache-Control: max-age=150
ETag: "698f7-N4B4o9RPah8r51cj0c5WsY0aKvg"
Vary: Origin, Accept-Encoding
Strict-Transport-Security: max-age=31536000; includeSubDomains
request-context: appId=cid-v1:
x-commit: c7fa6ad62b4e4646e990a17eceeb11a119726ea8
Content-Security-Polic